In [ ]:
import jupyterlab_nodeeditor as jlne
import ipywidgets
import time
from yggdrasil.runner import YggRunner
ne = jlne.NodeEditor()

# Change this to your model's filepath
model_filepath = "sample_test_models\model_trifecta.yml"
jlne.load_model(model_filepath, ne)

In [ ]:
# Create a runner for the model, adjust the filepath for wherever the model is located
runner = YggRunner(model_filepath)

# Create the callback class and update the first tab with live information

class VariableDisplay:
    # We add in filler labels to overwrite with actual data later since it is automated
    if len(ne.node_editor.nodes[0].display_element.children) <= 5:
        for i in range(len(list(runner.connectiondrivers))):
            ne.node_editor.nodes[0].display_element.children += (ipywidgets.Label("Initializing"),)
    
    # The initialization will have the label from ipywidgets but also the name of the variable and it's order in the list
    def __init__(self, elem_number, name):
        self.label = ipywidgets.Label()
        self.elem_number = elem_number
        self.name = name
    
    # Display data will always show the relevant information taken from the dictionary itself and use the default dictionary key as it's name
    # As of May 31st, 2022, the timesync bug still exists, so the time.sleep is necessary to see it run in realtime
    # Otherwise, it still works as intended
    # If you are only processing basic tings, you may opt to use the clean_bytes function with 'args[0].args' though it may be better to not use it as it may convolute the data being processed
    def display_data(self, *args, **kwargs):
        ne.node_editor.nodes[0].display_element.children[self.elem_number].value = f"{self.name}: {args[0].args}"
        time.sleep(1)

# We pull the exact drivers from the dictionary keys from the model itself here
for i, v in enumerate(list(runner.connectiondrivers)):
    display = VariableDisplay(i + 3, v.split(":")[1])
    runner.connectiondrivers[v]['callbacks'] = [display.display_data]

# Run the runner to see the data update in realtime
runner.run()